## 기존의 xlsx 파일을 csv로 변환

In [1]:
import pandas as pd
import math

song_df = pd.read_excel('../song_data/song_df.xlsx', index_col=0)
artist_df = pd.read_excel('../song_data/artist_df.xlsx', index_col=0)
album_df = pd.read_excel('../song_data/album_df.xlsx', index_col=0)
song_artist_df = pd.read_excel('../song_data/song_artist_df.xlsx', index_col=0)
song_album_df = pd.read_excel('../song_data/song_album_df.xlsx', index_col=0)
song_genre_df = pd.read_excel('../song_data/song_genre_df.xlsx', index_col=0)
lyrics_df = pd.read_excel('../song_data/song_lyrics_df.xlsx', index_col=0)

### 장르 테이블 (gnere_id, genre_name) 생성

In [4]:
genre_df = pd.DataFrame(columns=['genre_id', 'genre_name'])

genre_list = [
    [101, '발라드'],
    [102, '포크/블루스'],
    [103, '성인가요/트로트'],
    [104, '록/메탈'],
    [105, '국내영화'],
    [106, '댄스'],
    [107, '국내드라마'],
    [108, '뉴에이지'],
    [109, '클래식'],
    [110, '크로스오버'],
    [111, '교향/관현악'],
    [112, '랩/힙합'],
    [113, '-'],
    [114, '재즈'],
    [115, '보컬재즈'],
    [116, '인디음악'],
    [117, 'R&B/Soul'],
    [118, '애시드/퓨전/팝'],
    [119, '애니메이션/웹툰'],
    [120, 'POP'],
    [121, '일렉트로니카'],
    [122, 'J-POP'],
    [123, 'CCM'],
    [124, '국내CCM'],
    [125, '워십'],
    [126, '찬송가'],
    [127, '게임'],
    [128, '키즈'],
    [129, '만화'],
    [130, '국외영화'],
    [131, '국내뮤지컬']
]

for x,y in genre_list:
    genre_df.loc[len(genre_df)] = [x, y]

genre_df.to_csv('../song_data/genre.csv')

### 중복 값 제거 함수

In [41]:
lyrics_df.loc[16].lyrics

'미련없이 내뿜는 담배연기속에  \\n 아련히 떠오르는 그여인의 얼굴을 \\n 별마다 새겨보는 별마다 새겨보는  \\n 아~ 진고개 신사 \\n  \\n 헝클어진 머리를 쓸어 올리며 \\n 언젠가 불러주던 그여인의 노래를  \\n 소리없이 불러보던 소리없이 불러보던 \\n 아~ 진고개 신사 \n</div>'

In [53]:
song_lyrics_df = pd.DataFrame(columns=['song_id','title','lyrics'])

In [54]:
for i in range(len(song_df)):
    song_id, title = song_df.loc[i, ['song_id','title']]
    lyrics = lyrics_df[lyrics_df['song_id'] == song_id].lyrics.item()
    title = title.replace(',', '&')
    
    lyrics = lyrics.replace(',','_').replace('\n</div>', '').replace('</p>', '')
    if lyrics == 'None' or lyrics[:3] == '19금':
        lyrics = np.nan
    
    song_lyrics_df.loc[i, :] = [song_id, title, lyrics]

In [55]:
song_lyrics_df.to_excel('./song_lyrics_df.xlsx')
song_lyrics_df.to_csv('./song_lyrics_df.csv')

In [56]:
df = pd.read_csv('./song_lyrics_df.csv', index_col=0)
df

ParserError: Error tokenizing data. C error: Expected 4 fields in line 4378, saw 6


'미련없이 내뿜는 담배연기속에  \\n 아련히 떠오르는 그여인의 얼굴을 \\n 별마다 새겨보는 별마다 새겨보는  \\n 아~ 진고개 신사 \\n  \\n 헝클어진 머리를 쓸어 올리며 \\n 언젠가 불러주던 그여인의 노래를  \\n 소리없이 불러보던 소리없이 불러보던 \\n 아~ 진고개 신사 \n'